In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Downloading data

In [30]:
train_file = open("/home/pavel/MyDocs/Machine Learning/Simagin/lecture4/Kaggle_phonetics/train.txt","r")
train = train_file.read()

In [31]:
train = train.split('\n')
train = train[:-1]
print("Examples in test dataset:", len(train))
x_train = [data_example.split(' ')[0] for data_example in train]
y_train = [data_example.split(' ')[1] for data_example in train]

Examples in test dataset: 83194


In [32]:
print("Examples in x_train = ", len(x_train))
print("Examples in y_train = ", len(y_train))
print(x_train[:5])
print
print(y_train[:5])

Examples in x_train =  83194
Examples in y_train =  83194
['LEMIEUX', 'MINDING', 'STRIPED', 'KEN', 'CONFERENCE']
['L_AH_M_Y_UW', 'M_AY_N_D_IH_NG', 'S_T_R_AY_P_T', 'K_EH_N', 'K_AA_N_F_ER_AH_N_S']


In [38]:
print("Longest word in x_train -", max(x_train, key=len), "with len =", len(max(x_train, key=len)))
print("Longest word in y_train -", max(y_train, key=len), "with len =", len(max(y_train, key=len)))

Longest word in x_train - SUPERCALIFRAGILISTICEXPIALIDOCIOUS with len = 34
Longest word in y_train - S_UW_P_ER_K_AE_L_AH_F_R_AE_JH_AH_L_IH_S_T_IH_K_EH_K_S_P_IY_AE_L_AH_D_OW_SH_AH_S with len = 79


### First model architecture:
Represent each char as one-hot, use seq2seq without attention

### Second model architecture:
Represent each char as embedding, use seq2seq without attention

### Third model architecture:
Represent each char as embedding, use seq2seq with attention

### EXTRA Improvements:
Train biderectional rnn

### Preprocessing

In [ ]:
embeddingsFile = '/home/pavel/MyDocs/MachineLearning/Tinkoff/lecture6/char-embeddings.txt'

In [ ]:
class Vocab:
    def __init__(self, data):
        self.char2idx = {}
        self.idx2char = {}
        counter = 0
        for line in data:
            sym = line[0]
            if sym not in self.char2idx:
                self.char2idx[sym] = counter
                self.idx2char[counter] = sym
                counter += 1
    
    def update_sequence(self, sequence):
        newSequence = ''
        for char in sequence:
            if char in self.char2idx:
                newSequence += char
            else:
                newSequence += ' '
        newSequence = re.sub(' +',' ', newSequence)
        return newSequence

    def tokenize(self, sequence):
        sequence = self.update_sequence(sequence)
        return [self.char2idx[sym] for sym in sequence]
    
    def detokenize(self, sequence):
        return ''.join([self.idx2char[num] for num in sequence])